In [25]:
import gzip
import json
import pandas as pd
import os
import re

In [26]:
# What was the impact of the GE brand in the collection of tweets gathered in the past Olympic games?

tweets = []
with gzip.open("./olympics/2016-08-02-14.gz") as f:
    for line in f.readlines():
        tweets.append(json.loads(line))
        
dataset = pd.DataFrame(tweets)
dataset.info()

IOError: [Errno 2] No such file or directory: './olympics/2016-08-02-14.gz'

In [27]:
# Create additional columns
dataset['screen_name'] = dataset.user.map(lambda x: x['screen_name'].lower())
dataset['name'] = dataset.user.map(lambda x: x['name'].lower())
dataset['statuses_count'] = dataset.user.map(lambda x: x['statuses_count'])
dataset['followers_count'] = dataset.user.map(lambda x: x['followers_count'])
dataset['text_lower'] = dataset.text.apply(lambda x: x.lower())

In [28]:
ge_accounts = ['generalelectric', 
               'ge_reports', 
               'gedobrasil', 
               'ge_canada', 
               'gehealthcare', 
               'ge_digital', 
               'gedesign', 
               'ge_water', 
               'gecareeers_latam', 
               'geaviation', 
               'ge_appliances', 
               'gepublicaffairs', 
               'gecapital', 
               'ge_uk', 
               'geinfosec', 
               'ge_power', 
               'geresearch', 
               'ge_oilandgas', 
               'gelighting', 
               'ge_foundation', 
               'ge_gaspower']
dataset['retweeted_status'][2]

nan

In [29]:
# Volume
volume_dict = {}

# Tweets by GE
ge_tweets = dataset[dataset['screen_name'].isin(ge_accounts)]
volume_dict["tweets-by-ge"] = len(ge_tweets)

# Followers and tweet count per account
volume_dict['accounts'] = {}
ge_tweets_unique = ge_tweets.drop_duplicates(subset='screen_name')
ge_tweets_unique = ge_tweets_unique[['name', 'screen_name', 'statuses_count', 'followers_count']]
for index, row in ge_tweets_unique.iterrows():
    volume_dict['accounts'][row['screen_name']] = {
        'name': row['name'],
        'tweets': row['statuses_count'],
        'followers': row['followers_count'] 
    }
         
# Function to get counts per element
def get_counts(elem_list, volume_dict, dataset, key):
    current_count = 0
    total_count = 0
    for elem in elem_list:
        current_count = len(dataset[dataset['text_lower'].str.contains(elem)])
        volume_dict[key][elem] = current_count
        total_count += current_count
    return total_count

# Function to get counts for a regex
def get_regex_counts(volume_dict, dataset, key, regex, regexstr):
    count = len(dataset[dataset['text_lower'].str.match(regex)])
    volume_dict[key][regexstr] = count
    return count

# Dataset excluding GE's accounts
exclude_ge = dataset[~dataset['screen_name'].isin(ge_accounts)]
    
# Mentions of GE accounts
volume_dict['mentions'] = {}
ge_mentions = list(ge_accounts)
ge_mentions = ['@{0}'.format(account) for account in ge_mentions]
ge_regex = re.compile('@ge | @ge$')
total_mentions = get_counts(ge_mentions, volume_dict, exclude_ge, 'mentions')
total_mentions += get_regex_counts(volume_dict, exclude_ge, 'mentions', ge_regex, "@ge")
volume_dict['total_mentions'] = total_mentions

# Hashtags of GE
volume_dict['hashtags'] = {}
ge_hashtags = list(ge_accounts)
ge_hashtags = ['#{0}'.format(account) for account in ge_mentions]
ge_hashtags.append("#general_electric")
ge_hashtag_regex = re.compile('#ge | #ge$')
ge_dollar_regex = re.compile('\$ge | \$ge$')
total_hashtags = get_counts(ge_hashtags, volume_dict, exclude_ge, 'hashtags')
total_hashtags = get_regex_counts(volume_dict, exclude_ge, 'hashtags', ge_dollar_regex, "$ge")
total_hashtags = get_regex_counts(volume_dict, exclude_ge, 'hashtags', ge_hashtag_regex, "#ge")
volume_dict['total_hashtags'] = total_hashtags

# GE mentioned in text
volume_dict['text_mentions'] = {}
ge_text = ['general electric', 'generalelectric']
ge_website_regex = re.compile('ge.com | ge.com$')
total_hashtags = get_counts(ge_text, volume_dict, exclude_ge, 'text_mentions')
total_hashtags += get_regex_counts(volume_dict, exclude_ge, 'text_mentions', ge_website_regex, "ge.com")
volume_dict['total_text_mentions'] = total_hashtags

volume_dict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6261 entries, 0 to 6260
Data columns (total 37 columns):
contributors                 0 non-null object
coordinates                  15 non-null object
created_at                   6261 non-null object
entities                     6261 non-null object
extended_entities            2388 non-null object
favorite_count               6261 non-null int64
favorited                    6261 non-null bool
filter_level                 6261 non-null object
geo                          15 non-null object
id                           6261 non-null int64
id_str                       6261 non-null object
in_reply_to_screen_name      213 non-null object
in_reply_to_status_id        147 non-null float64
in_reply_to_status_id_str    147 non-null object
in_reply_to_user_id          213 non-null float64
in_reply_to_user_id_str      213 non-null object
is_quote_status              6261 non-null bool
lang                         6261 non-null object
place    

In [31]:
# Reach

# All tweets made by GE
ge_tweets = dataset[dataset['screen_name'].isin(ge_accounts)]
ge_tweets['reach'] = 0
ge_tweets = ge_tweets['id', 'retweet_count', 'reach', 'screen_name']

# GE retweeters
exclude_ge = dataset[~dataset['screen_name'].isin(ge_accounts)]
ge_retweets = ['^rt @{0}'.format(account) for account in ge_accounts]
ge_retweeters = dataset[dataset['text_lower'].str.contains('|'.join(ge_retweets))]

# For every GE tweet, we need to compute the reach.
# for tweeter in ge_retweeters:
#     ge_tweets[tweeter[retweet_status[id]]]['reach'] +

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 38 columns):
contributors                 0 non-null object
coordinates                  0 non-null object
created_at                   0 non-null object
entities                     0 non-null object
extended_entities            0 non-null object
favorite_count               0 non-null int64
favorited                    0 non-null bool
filter_level                 0 non-null object
geo                          0 non-null object
id                           0 non-null int64
id_str                       0 non-null object
in_reply_to_screen_name      0 non-null object
in_reply_to_status_id        0 non-null float64
in_reply_to_status_id_str    0 non-null object
in_reply_to_user_id          0 non-null float64
in_reply_to_user_id_str      0 non-null object
is_quote_status              0 non-null bool
lang                         0 non-null object
place                        0 non-null object
possibly_sensitive

In [ ]:
# Sentiment

In [ ]:
# Audience Characteristics

# Lang
# Location

# Most active participants
# Most influential participants
# Most influential content